In [54]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [23]:
def detailinforequest(lan,hs):
    # URL của API
    url = 'https://unipass.customs.go.kr/clip/hsinfosrch/retrieveManlInfoSrch.do'

    # Headers
    headers = {
        'Accept': 'text/html, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Cookie': 'WMONID=N_4A3dINH0I; JSESSIONID=0001O2y8S-yyQQ_lga-knblPV6Lb2QN6662ivRFJ_jK2FPRlLOTKNEsnuhZe_QxBEKq-8U03_kah8EgUgb-_Exa4dmevvhJHEKeNuIq9prRxpJht6ugFASENgz3p_AdMPw4_:eul21',
        'DNT': '1',
        'Origin': 'https://unipass.customs.go.kr',
        'Referer': 'https://unipass.customs.go.kr/clip/index.do',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'isAjax': 'true',
        'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"'
    }

    # Dữ liệu trong body của yêu cầu
    payload = {
        'cntyCd': 'VN',
        'lngTpcd': lan,
        'compareCrrspndNation': '',
        'sctYear': '20240101',
        'hstdYear': '20240101',
        'hsfdYear': '',
        'manlOrgnTpcd': '01',
        'aplyYy': '2024',
        'tabTpcd': '3',
        'hsGnprSrno': '1',
        'searchVal': hs,
        'euTp': '',
        'tmpManlOrgnTpcd': ''
    }


    # Gửi yêu cầu POST với headers và dữ liệu body
    response = requests.post(url, headers=headers, data=payload,timeout=60)

    # Kiểm tra phản hồi từ API
    if response.status_code == 200:
        return response.json()['resultHsSgnVo']
    else:
        print(f"HS {hs} Lỗi")
        return None

In [36]:
def extract_data_from_json(lan,json):
    result  = {}
    result[f'{lan}section']           = json['sctCd']
    result[f'{lan}section_note']      = json['sctManl']
    result[f'{lan}chapter']           = json['hstdCd']
    result[f'{lan}chapter_notes']     = json['hstdManl']
    result[f'{lan}heading']           = json['hsfdCd']
    result[f'{lan}heading_notes']     = json['hsfdManl']
    return result

In [ ]:
all_result = []

In [49]:
file_path = "heading_list.txt"

# Mở file và đọc từng dòng
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

# Loại bỏ ký tự xuống dòng (\n) ở cuối mỗi dòng
heading_list = [line.strip() for line in lines]

In [50]:
len(heading_list)

1268

In [ ]:

while True: 
    try:
        for hs in heading_list[725:]:
            hs_str = f"{hs:04}"
            print(hs_str)
            json_response = detailinforequest('eng',hs_str)
            if json_response:
                eng_result = extract_data_from_json('en',json_response)
                ko_result = extract_data_from_json('ko',detailinforequest('kor',hs_str))
                en_ko_result = eng_result | ko_result
                print(en_ko_result)
                all_result.append(en_ko_result)
    except Exception as e:
        time.sleep(5)  # Đợi 5 giây trước khi thử lại

In [ ]:
import pandas as pd
def clean_data(all_result_df):
    # Drop duplicate rows across all columns
    all_result_df = all_result_df.drop_duplicates()
    return all_result_df

# Loaded variable 'all_result' from kernel state
all_result_df = pd.DataFrame(all_result)

all_result_df_clean = clean_data(all_result_df.copy())

,ensection,ensection_note,enchapter,enchapter_notes,enheading,enheading_notes,kosection,kosection_note,kochapter,kochapter_notes,koheading,koheading_notes
0,01,Section I\n\nLIVE ANIMALS; \nANIMAL PRODUCTS\n...,01,Chapter 1\n\n\nLive animals\n\n\nNote. \n\n1.-...,0101,"01.01 Live horses, asses, mules and hinnies ...",01,제 1 부\n\n살아 있는 동물과 동물성 생산품\n\n주: \n\n1. 이 부에 열...,01,제1류\n살아 있는 동물\n\n주: \n\n1. 이 류에는 다음 각 목의 것을 제외...,0101,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n\n- 말\n\n0101.21...
1,01,Section I\n\nLIVE ANIMALS; \nANIMAL PRODUCTS\n...,01,Chapter 1\n\n\nLive animals\n\n\nNote. \n\n1.-...,0102,01.02 Live bovine animals (+).\n\n - Cattle ...,01,제 1 부\n\n살아 있는 동물과 동물성 생산품\n\n주: \n\n1. 이 부에 열...,01,제1류\n살아 있는 동물\n\n주: \n\n1. 이 류에는 다음 각 목의 것을 제외...,0102,01.02 - 살아 있는 소(+)\n\n- 축우(畜牛)\n \n0102.21 - 번...
2,01,Section I\n\nLIVE ANIMALS; \nANIMAL PRODUCTS\n...,01,Chapter 1\n\n\nLive animals\n\n\nNote. \n\n1.-...,0103,01.03 - Live swine (+).\n\n0103.10 - Pure bred...,01,제 1 부\n\n살아 있는 동물과 동물성 생산품\n\n주: \n\n1. 이 부에 열...,01,제1류\n살아 있는 동물\n\n주: \n\n1. 이 류에는 다음 각 목의 것을 제외...,0103,01.03 - 살아 있는 돼지(+)\n\n0103.10 - 번식용\n\n- 기타\n...
3,01,Section I\n\nLIVE ANIMALS; \nANIMAL PRODUCTS\n...,01,Chapter 1\n\n\nLive animals\n\n\nNote. \n\n1.-...,0104,01.04 - Live sheep and goats. \n\n0104.10 - Sh...,01,제 1 부\n\n살아 있는 동물과 동물성 생산품\n\n주: \n\n1. 이 부에 열...,01,제1류\n살아 있는 동물\n\n주: \n\n1. 이 류에는 다음 각 목의 것을 제외...,0104,01.04 - 살아 있는 면양과 염소\n\n0104.10 - 면양\n\n0104.2...
4,01,Section I\n\nLIVE ANIMALS; \nANIMAL PRODUCTS\n...,01,Chapter 1\n\n\nLive animals\n\n\nNote. \n\n1.-...,0105,"01.05 - Live poultry, that is to say, fowls of...",01,제 1 부\n\n살아 있는 동물과 동물성 생산품\n\n주: \n\n1. 이 부에 열...,01,제1류\n살아 있는 동물\n\n주: \n\n1. 이 류에는 다음 각 목의 것을 제외...,0105,01.05 - 살아 있는 가금(家禽)류[닭(갈루스 도메스티쿠스(Gallus dome...


In [58]:
all_result_df_clean.to_excel('ko_en_section_chapter_heading_des_notes.xlsx')